In [1]:
from datasets import load_from_disk

dataset_dir = "D:/dolly15k/processed"  # 事前にデータセットを保存したディレクトリを指定
processed_dataset = load_from_disk(dataset_dir)

processed_dataset



DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 13509
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1502
    })
})

In [13]:
train_dataset = processed_dataset['train']
for i in range(5):
    print(f"Sample {i}, input_ids length: {len(train_dataset[i]['input_ids'])}")

Sample 0, input_ids length: 209
Sample 1, input_ids length: 29
Sample 2, input_ids length: 30
Sample 3, input_ids length: 101
Sample 4, input_ids length: 65


In [22]:
import dotenv
import os
from transformers import AutoTokenizer, DataCollatorForLanguageModeling, DataCollatorForSeq2Seq
from torch.utils.data import DataLoader

# .envファイルの読み込み
dotenv.load_dotenv(".env")
 
# トークナイザの読み込み
token = os.environ.get("HF_TOKEN")
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(
    model_name, use_fast=True, token=token
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# データコラレータの定義
# data_collator = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer, 
#     mlm=False,
#     pad_to_multiple_of=8
# )

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    pad_to_multiple_of=8,
    label_pad_token_id=-100
)

# データローダの定義
train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=data_collator,
    batch_size=2
)

for batch in train_dataloader:    
    x = batch['input_ids']
    y = batch['labels']
    
    print(x.shape, y.shape)
    break

torch.Size([2, 112]) torch.Size([2, 112])
